In [1]:
# wrangling
import numpy as np
import pandas as pd
import feather
pd.options.display.max_columns = 200

# preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

# model
from keras.optimizers import Adam
from keras.models import Sequential  
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
from keras import backend as K

# metrics
from sklearn.metrics import accuracy_score, log_loss

# others
import os
import random as rn
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


# load data

In [2]:
def load_rnn_data(path, window, predict_ts, isdim3=True, geo_col=["geoid10_tract"], y_cols=["crime"]):
    """
    y_cols: ["crime"] or ["incident_type_0", "incident_type_1", "incident_type_2"]
    geo_col: ["geoid10_tract"] or ["geoid10_block"]
    return y_all and x_all of given path
    """
    # load data
    df = feather.read_dataframe(path)
    df.sort_values(by=["datetime", "geoid10_tract"], inplace=True)
    df.set_index("datetime", inplace=True)

    # input columns
    x_cols = list(df.drop(y_cols + geo_col, axis=1).columns)

    # group by geoid
    geo_grs = df.groupby(by=geo_col)

    # arrayes to store x and y
    # (no of timesteps, window size, no of tracts,  no of features, )
    n_timesteps = int(len(df) / len(geo_grs)) - window - predict_ts + 1
    x_all = np.empty(shape=(n_timesteps, window, len(geo_grs), len(x_cols + y_cols)))

    # (output size, no of tracts, no of outputs)
    y_all = np.empty(shape=(n_timesteps, len(geo_grs), len(y_cols)))

    # to store geo_ids and y_all's datetime
    geo_ids = []

    y_datetime = df.index.unique()[window + predict_ts - 1:]

    for i, (geo_id, gr) in enumerate(tqdm(geo_grs)):
        geo_ids.append(geo_id)
        x_values = gr[y_cols + x_cols].values
        y_values = gr[y_cols].values

        for j in range(window, len(gr) - predict_ts + 1):
            # generate x_all
            x_all[j - window, :, i, :] = x_values[j - window:j, :]
            y_all[j - window, i, :] = y_values[j + predict_ts - 1, :]

    if isdim3:
        x_all = np.reshape(x_all,
                           newshape=(x_all.shape[0], x_all.shape[1], x_all.shape[2] * x_all.shape[3]))
        y_all = np.reshape(y_all,
                           newshape=(y_all.shape[0], y_all.shape[1] * y_all.shape[2]))

    return x_all, y_all, geo_ids, y_datetime

In [3]:
# set configuration
path = "./features/features_binary_tract_2H.feather"
window = 12
predict_ts = 1  # how many timesteps future does the model predict? 

In [4]:
# load x and y
x_all, y_all, geo_ids, y_datetime = load_rnn_data(path=path,
                                                  window=window,
                                                  predict_ts=predict_ts,
                                                  isdim3=True,
                                                  geo_col=["geoid10_tract"],
                                                  y_cols=["crime"])

In [5]:
print(x_all.shape)
print(y_all.shape)
print(len(geo_ids))  # to convert model output later
print(len(y_datetime))  # to convert model output later

(18600, 12, 1560)
(18600, 195)
195
18600


# preprocessing

In [6]:
# # scaling
# scaler = MinMaxScaler()
# x_all = scaler.fit_transform(x_all)

# modeling

In [7]:
def time_series_cv(x_all, y_all, n_splits=5, model=None, fit_params=None, baseline=False):
    """
    :param baseline: True or False (defualt: False)
    :return: train and test scores and prediction of y on test data
    """

    # prepare dictionary to store scores
    train_scores = {}
    metrics = ["acc", "log_loss"]
    for metric in metrics:
        train_scores[metric] = []
    test_scores = deepcopy(train_scores)

    # prepare dictionary to store predictions
    y_test_probs = np.zeros_like(y_all)

    # time series split
    tss = TimeSeriesSplit(n_splits=n_splits)

    for split, (train_idx, test_idx) in enumerate(tss.split(x_all, y_all)):

        print("---------- split {0} ----------".format(split))
        print("[{0:%H:%M:%S}] train_index:{1}~{2} test_index:{3}~{4}".format(
            datetime.now(), train_idx[0], train_idx[-1], test_idx[0], test_idx[-1]))

        # create train and test set
        x_train = x_all[:train_idx[-1]]
        y_train = y_all[:train_idx[-1]]
        x_test = x_all[test_idx[0]:test_idx[-1]]
        y_test = y_all[test_idx[0]:test_idx[-1]]

        if baseline:
            # return 0 for all predicted probabiliby
            y_train_prob = np.zeros_like(y_train)
            y_test_prob = np.zeros_like(y_test)

            # return 0 for all binary predictions
            y_train_pred = np.zeros_like(y_train)
            y_test_pred = np.zeros_like(y_test)

        else:            
            # train
            model.fit(x_train, y_train, **fit_params)

            # predict
            y_train_prob = model.predict(x_train)
            y_test_prob = model.predict(x_test)

            # convert form probability to binary
            y_train_pred = np.around(y_train_prob)
            y_test_pred = np.around(y_test_prob)

        # store test prediction
        y_test_probs[test_idx[0]:test_idx[-1]] = y_test_prob

        # calculate metrics
        train_log_loss = log_loss(y_train.flatten(), y_train_prob.flatten())
        test_log_loss = log_loss(y_test.flatten(), y_test_prob.flatten())
        train_acc = accuracy_score(y_train.flatten(), y_train_pred.flatten())
        test_acc = accuracy_score(y_test.flatten(), y_test_pred.flatten())

        # store scores
        train_scores["log_loss"].append(train_log_loss)
        test_scores["log_loss"].append(test_log_loss)
        train_scores["acc"].append(train_acc)
        test_scores["acc"].append(test_acc)

        print("[{0:%H:%M:%S}] train_log_loss:{1} test_log_loss:{2}".format(
            datetime.now(), train_log_loss, test_log_loss))
        print("[{0:%H:%M:%S}] train_acc:{1} test_acc:{2}\n".format(
            datetime.now(), train_acc, test_acc))

        # convert to dataframe
        train_scores_df = pd.DataFrame(train_scores)
        test_scores_df = pd.DataFrame(test_scores)

    return train_scores_df, test_scores_df, y_test_probs

In [8]:
class MyLSTM():
    """
    class to input into function time_series_cv
    """
    def __init__(self, units=100,
                 dropout_rate=0.2, activation="sigmoid",
                 optimizer="adam", loss="binary_crossentropy"):
        """
        define LSTM model with given arguments
        """
        self.units = units
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss

    def fit(self, x_train, y_train, epochs=200, early_stopping_patience=5,
            batch_size=128, validation_split=0.1, train_shuffle=False):
        """
        fit train data
        """
        self.model = Sequential()
        self.model.add(LSTM(units=self.units, input_shape=(x_train.shape[1], x_train.shape[2])))
        self.model.add(Dropout(self.dropout_rate))
        self.model.add(Dense(units=y_train.shape[1], activation=self.activation))
        self.model.compile(optimizer=self.optimizer, loss=self.loss, metrics=["acc"])

        earlystopping = EarlyStopping(monitor="val_loss", patience=early_stopping_patience)
        self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                       validation_split=validation_split, shuffle=train_shuffle,
                       callbacks=[earlystopping])
        
    def predict(self, x_test):
        """
        return predicted values of x_test
        """
        y_prob = self.model.predict(x_test)
        return y_prob

In [9]:
# parameters at model definition
def_params = {
    "units":100,
    "dropout_rate":0.159961594635,
    "activation":"sigmoid",
    "optimizer":Adam(lr=1.77547438185e-05),
    "loss":"binary_crossentropy"
}

# parameters at model training
fit_params = {
    "epochs":500,
    "early_stopping_patience":5,
    "batch_size":128,
    "validation_split":0.1,
    "train_shuffle":False
}

# number of time series cv splits
n_splits = 5

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# set seed
np.random.seed(0)
os.environ["PYTHONHASHSEED"] = "0"
rn.seed(0)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [11]:
# Baseline --> Assign all observations to = 0 (no crime)

model = MyLSTM(**def_params)

train_scores_df, test_scores_df, y_test_probs = time_series_cv(x_all,
                                                               y_all,
                                                               n_splits=n_splits,
                                                               model=model,
                                                               fit_params=fit_params,
                                                               baseline=True)

---------- split 0 ----------
[16:57:27] train_index:0~3099 test_index:3100~6199
[16:57:27] train_log_loss:3.7914609505286054 test_log_loss:2.744732641319857
[16:57:27] train_acc:0.8902259620555845 test_acc:0.9205318506383366

---------- split 1 ----------
[16:57:27] train_index:0~6199 test_index:6200~9299
[16:57:28] train_log_loss:3.267798179800001 test_log_loss:2.880874763878297
[16:57:28] train_acc:0.9053875521692911 test_acc:0.9165901324662216

---------- split 2 ----------
[16:57:28] train_index:0~9299 test_index:9300~12399
[16:57:29] train_log_loss:3.138861934241693 test_log_loss:3.137727282256813
[16:57:29] train_acc:0.9091206388335111 test_acc:0.9091534903732387

---------- split 3 ----------
[16:57:29] train_index:0~12399 test_index:12400~15499
[16:57:31] train_log_loss:3.1386966000817376 test_log_loss:3.453963371306283
[16:57:31] train_acc:0.909125425747734 test_acc:0.8999975178097153

---------- split 4 ----------
[16:57:31] train_index:0~15499 test_index:15500~18599
[16:57:

In [12]:
# train metrics for baseline
train_scores_df.to_csv("./results/train_scores_base.csv", index=False)
train_scores_df

,acc,log_loss
0,0.890226,3.791461
1,0.905388,3.267798
2,0.909121,3.138862
3,0.909125,3.138697
4,0.907303,3.201634


In [13]:
# test metrics for baseline
test_scores_df.to_csv("./results/test_scores_base.csv", index=False)
test_scores_df

,acc,log_loss
0,0.920532,2.744733
1,0.916590,2.880875
2,0.909153,3.137727
3,0.899998,3.453963
4,0.894838,3.632171


In [14]:
# Actual Model --> Assign all observations to prediction

# define model
model = MyLSTM(**def_params)

# time series cross validation
train_scores_df, test_scores_df, y_test_probs = time_series_cv(x_all,
                                                               y_all,
                                                               n_splits=n_splits,
                                                               model=model,
                                                               fit_params=fit_params,
                                                               baseline=False)

---------- split 0 ----------
[16:57:33] train_index:0~3099 test_index:3100~6199
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 2789 samples, validate on 310 samples
Epoch 1/500
2789/2789 [==============================] - 6s 2ms/step - loss: 0.7049 - acc: 0.5343 - val_loss: 0.6954 - val_acc: 0.5656
Epoch 2/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.7000 - acc: 0.5417 - val_loss: 0.6920 - val_acc: 0.5705
Epoch 3/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.6971 - acc: 0.5470 - val_loss: 0.6886 - val_acc: 0.5755
Epoch 4/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.6942 - acc: 0.5519 - val_loss: 0.6852 - val_acc: 0.5804
Epoch 5/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.6911 - acc: 0.5580 - val_loss: 0.6819 - val_acc: 0.5901
Epoch 6/500
2789/2789 [===========

2789/2789 [==============================] - 5s 2ms/step - loss: 0.4759 - acc: 0.8723 - val_loss: 0.4222 - val_acc: 0.9378
Epoch 116/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4743 - acc: 0.8731 - val_loss: 0.4205 - val_acc: 0.9426
Epoch 117/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4742 - acc: 0.8733 - val_loss: 0.4189 - val_acc: 0.9475
Epoch 118/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4720 - acc: 0.8742 - val_loss: 0.4173 - val_acc: 0.9475
Epoch 119/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4717 - acc: 0.8745 - val_loss: 0.4157 - val_acc: 0.9475
Epoch 120/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4699 - acc: 0.8750 - val_loss: 0.4142 - val_acc: 0.9475
Epoch 121/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4688 - acc: 0.8755 - val_loss: 0.4126 - val_acc: 0.9475
Epoch 122/500
2789/2789 [==============================

Epoch 175/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4142 - acc: 0.8859 - val_loss: 0.3409 - val_acc: 0.9525
Epoch 176/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4128 - acc: 0.8859 - val_loss: 0.3397 - val_acc: 0.9525
Epoch 177/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4119 - acc: 0.8861 - val_loss: 0.3386 - val_acc: 0.9525
Epoch 178/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4115 - acc: 0.8860 - val_loss: 0.3375 - val_acc: 0.9525
Epoch 179/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4109 - acc: 0.8861 - val_loss: 0.3364 - val_acc: 0.9525
Epoch 180/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4099 - acc: 0.8862 - val_loss: 0.3354 - val_acc: 0.9525
Epoch 181/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4089 - acc: 0.8863 - val_loss: 0.3343 - val_acc: 0.9525
Epoch 182/500
2789/2789 [================

Epoch 235/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3744 - acc: 0.8867 - val_loss: 0.2852 - val_acc: 0.9525
Epoch 236/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3739 - acc: 0.8868 - val_loss: 0.2845 - val_acc: 0.9525
Epoch 237/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3733 - acc: 0.8868 - val_loss: 0.2837 - val_acc: 0.9525
Epoch 238/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3724 - acc: 0.8867 - val_loss: 0.2830 - val_acc: 0.9525
Epoch 239/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3718 - acc: 0.8867 - val_loss: 0.2822 - val_acc: 0.9525
Epoch 240/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3714 - acc: 0.8868 - val_loss: 0.2815 - val_acc: 0.9525
Epoch 241/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3711 - acc: 0.8868 - val_loss: 0.2808 - val_acc: 0.9525
Epoch 242/500
2789/2789 [================

Epoch 295/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3500 - acc: 0.8867 - val_loss: 0.2479 - val_acc: 0.9525
Epoch 296/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3495 - acc: 0.8867 - val_loss: 0.2474 - val_acc: 0.9525
Epoch 297/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3492 - acc: 0.8868 - val_loss: 0.2469 - val_acc: 0.9525
Epoch 298/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3488 - acc: 0.8867 - val_loss: 0.2464 - val_acc: 0.9525
Epoch 299/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3486 - acc: 0.8868 - val_loss: 0.2459 - val_acc: 0.9525
Epoch 300/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3485 - acc: 0.8868 - val_loss: 0.2454 - val_acc: 0.9525
Epoch 301/500
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3483 - acc: 0.8868 - val_loss: 0.2449 - val_acc: 0.9525
Epoch 302/500
2789/2789 [================

...

11159/11159 [==============================] - 21s 2ms/step - loss: 0.3083 - acc: 0.9096 - val_loss: 0.2977 - val_acc: 0.9141
Epoch 73/500
11159/11159 [==============================] - 21s 2ms/step - loss: 0.3072 - acc: 0.9096 - val_loss: 0.2964 - val_acc: 0.9141
Epoch 74/500
11159/11159 [==============================] - 21s 2ms/step - loss: 0.3056 - acc: 0.9095 - val_loss: 0.2952 - val_acc: 0.9141
Epoch 75/500
11159/11159 [==============================] - 22s 2ms/step - loss: 0.3046 - acc: 0.9096 - val_loss: 0.2940 - val_acc: 0.9141
Epoch 76/500
11159/11159 [==============================] - 23s 2ms/step - loss: 0.3035 - acc: 0.9096 - val_loss: 0.2928 - val_acc: 0.9141
Epoch 77/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.3026 - acc: 0.9096 - val_loss: 0.2916 - val_acc: 0.9141
Epoch 78/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.3013 - acc: 0.9096 - val_loss: 0.2905 - val_acc: 0.9141
Epoch 79/500
11159/11159 [==============

Epoch 131/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.2760 - acc: 0.9096 - val_loss: 0.2638 - val_acc: 0.9141
Epoch 132/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.2758 - acc: 0.9096 - val_loss: 0.2637 - val_acc: 0.9141
Epoch 133/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.2756 - acc: 0.9096 - val_loss: 0.2635 - val_acc: 0.9141
Epoch 134/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.2754 - acc: 0.9096 - val_loss: 0.2634 - val_acc: 0.9141
Epoch 135/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.2753 - acc: 0.9096 - val_loss: 0.2633 - val_acc: 0.9141
Epoch 136/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.2752 - acc: 0.9096 - val_loss: 0.2631 - val_acc: 0.9141
Epoch 137/500
11159/11159 [==============================] - 20s 2ms/step - loss: 0.2752 - acc: 0.9096 - val_loss: 0.2630 - val_acc: 0.9141
Epoch 138/500
11159/

11159/11159 [==============================] - 27s 2ms/step - loss: 0.2733 - acc: 0.9096 - val_loss: 0.2607 - val_acc: 0.9141
Epoch 248/500
11159/11159 [==============================] - 24s 2ms/step - loss: 0.2731 - acc: 0.9096 - val_loss: 0.2607 - val_acc: 0.9141
Epoch 249/500
11159/11159 [==============================] - 28s 2ms/step - loss: 0.2731 - acc: 0.9096 - val_loss: 0.2607 - val_acc: 0.9141
Epoch 250/500
11159/11159 [==============================] - 23s 2ms/step - loss: 0.2730 - acc: 0.9096 - val_loss: 0.2607 - val_acc: 0.9141
Epoch 251/500
11159/11159 [==============================] - 24s 2ms/step - loss: 0.2732 - acc: 0.9096 - val_loss: 0.2607 - val_acc: 0.9141
Epoch 252/500
11159/11159 [==============================] - 23s 2ms/step - loss: 0.2732 - acc: 0.9096 - val_loss: 0.2607 - val_acc: 0.9141
Epoch 253/500
11159/11159 [==============================] - 23s 2ms/step - loss: 0.2734 - acc: 0.9096 - val_loss: 0.2607 - val_acc: 0.9141
Epoch 254/500
11159/11159 [=======

Epoch 23/500
13949/13949 [==============================] - 32s 2ms/step - loss: 0.3953 - acc: 0.9087 - val_loss: 0.3951 - val_acc: 0.9026
Epoch 24/500
13949/13949 [==============================] - 26s 2ms/step - loss: 0.3904 - acc: 0.9088 - val_loss: 0.3903 - val_acc: 0.9026
Epoch 25/500
13949/13949 [==============================] - 32s 2ms/step - loss: 0.3853 - acc: 0.9088 - val_loss: 0.3857 - val_acc: 0.9026
Epoch 26/500
13949/13949 [==============================] - 29s 2ms/step - loss: 0.3807 - acc: 0.9089 - val_loss: 0.3813 - val_acc: 0.9026
Epoch 27/500
13949/13949 [==============================] - 25s 2ms/step - loss: 0.3764 - acc: 0.9088 - val_loss: 0.3771 - val_acc: 0.9026
Epoch 28/500
13949/13949 [==============================] - 26s 2ms/step - loss: 0.3722 - acc: 0.9088 - val_loss: 0.3730 - val_acc: 0.9026
Epoch 29/500
13949/13949 [==============================] - 25s 2ms/step - loss: 0.3680 - acc: 0.9089 - val_loss: 0.3691 - val_acc: 0.9026
Epoch 30/500
13949/13949 [=

Epoch 82/500
13949/13949 [==============================] - 39s 3ms/step - loss: 0.2807 - acc: 0.9088 - val_loss: 0.2862 - val_acc: 0.9026
Epoch 83/500
13949/13949 [==============================] - 27s 2ms/step - loss: 0.2801 - acc: 0.9089 - val_loss: 0.2859 - val_acc: 0.9026
Epoch 84/500
13949/13949 [==============================] - 27s 2ms/step - loss: 0.2798 - acc: 0.9089 - val_loss: 0.2856 - val_acc: 0.9026
Epoch 85/500
13949/13949 [==============================] - 32s 2ms/step - loss: 0.2796 - acc: 0.9088 - val_loss: 0.2853 - val_acc: 0.9026
Epoch 86/500
13949/13949 [==============================] - 25s 2ms/step - loss: 0.2791 - acc: 0.9089 - val_loss: 0.2850 - val_acc: 0.9026
Epoch 87/500
13949/13949 [==============================] - 25s 2ms/step - loss: 0.2789 - acc: 0.9089 - val_loss: 0.2847 - val_acc: 0.9026
Epoch 88/500
13949/13949 [==============================] - 25s 2ms/step - loss: 0.2785 - acc: 0.9089 - val_loss: 0.2845 - val_acc: 0.9026
Epoch 89/500
13949/13949 [=

# Result

In [15]:
# train metrics
train_scores_df.to_csv("./results/train_scores.csv", index=False)
train_scores_df

,acc,log_loss
0,0.893426,0.308521
1,0.906527,0.278036
2,0.910056,0.270104
3,0.910147,0.269837
4,0.908355,0.273060


In [16]:
# test metrics
test_scores_df.to_csv("./results/test_scores.csv", index=False)
test_scores_df

,acc,log_loss
0,0.918609,0.260748
1,0.916545,0.255125
2,0.910434,0.269369
3,0.901172,0.286154
4,0.894921,0.298277


In [17]:
# predicted probability of crime occurance
y_probs_df = pd.DataFrame(y_test_probs, index=y_datetime, columns=geo_ids)
y_probs_df = y_probs_df.loc[(y_probs_df != 0).any(axis=1), :]
y_probs_df.to_csv("./results/y_probs.csv", index=True)
y_probs_df.head()

,06075010100,06075010200,06075010300,06075010400,06075010500,06075010600,06075010700,06075010800,06075010900,06075011000,06075011100,06075011200,06075011300,06075011700,06075011800,06075011901,06075011902,06075012000,06075012100,06075012201,06075012202,06075012301,06075012302,06075012401,06075012402,06075012501,06075012502,06075012601,06075012602,06075012700,06075012800,06075012901,06075012902,06075013000,06075013101,06075013102,06075013200,06075013300,06075013400,06075013500,06075015100,06075015200,06075015300,06075015400,06075015500,06075015600,06075015700,06075015801,06075015802,06075015900,06075016000,06075016100,06075016200,06075016300,06075016400,06075016500,06075016600,06075016700,06075016801,06075016802,06075016900,06075017000,06075017101,06075017102,06075017601,06075017700,06075017801,06075017802,06075017902,06075018000,06075020100,06075020200,06075020300,06075020401,06075020402,06075020500,06075020600,06075020700,06075020800,06075020900,06075021000,06075021100,06075021200,06075021300,06075021400,06075021500,06075021600,06075021700,06075021800,06075022600,06075022702,06075022704,06075022801,06075022802,06075022803,06075022901,06075022902,06075022903,06075023001,06075023003,06075023102,06075023103,06075023200,06075023300,06075023400,06075025100,06075025200,06075025300,06075025401,06075025402,06075025403,06075025500,06075025600,06075025701,06075025702,06075025800,06075025900,06075026001,06075026002,06075026003,06075026004,06075026100,06075026200,06075026301,06075026302,06075026303,06075026401,06075026402,06075026403,06075026404,06075030101,06075030102,06075030201,06075030202,06075030301,06075030302,06075030400,06075030500,06075030600,06075030700,06075030800,06075030900,06075031000,06075031100,06075031201,06075031202,06075031301,06075031302,06075031400,06075032601,06075032602,06075032700,06075032801,06075032802,06075032901,06075032902,06075033000,06075033100,06075033201,06075033203,06075033204,06075035100,06075035201,06075035202,06075035300,06075035400,06075040100,06075040200,06075042601,06075042602,06075042700,06075042800,06075045100,06075045200,06075047600,06075047701,06075047702,06075047801,06075047802,06075047901,06075047902,06075060100,06075060400,06075060502,06075060700,06075061000,06075061100,06075061200,06075061400,06075061500,06075980200,06075980300,06075980501,06075980600,06075980900
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-09-17 08:00:00,0.387637,0.241744,0.098925,0.095542,0.292688,0.127813,0.185343,0.05997,0.095403,0.100532,0.201888,0.090691,0.07577,0.542401,0.057486,0.050868,0.101254,0.18622,0.153662,0.164475,0.218936,0.248881,0.178816,0.277531,0.451848,0.359447,0.225989,0.062736,0.094343,0.086376,0.17519,0.07913,0.094767,0.123563,0.092806,0.072251,0.081813,0.065912,0.061012,0.087861,0.14852,0.116283,0.059592,0.101352,0.163449,0.082159,0.16789,0.147182,0.098014,0.225008,0.107451,0.214664,0.190793,0.100662,0.133423,0.133659,0.214096,0.111621,0.097745,0.124948,0.158183,0.073988,0.059291,0.070247,0.644698,0.365396,0.185017,0.407634,0.070816,0.637707,0.440635,0.150933,0.132466,0.076848,0.055367,0.121466,0.168135,0.144559,0.22112,0.194331,0.073108,0.06429,0.055051,0.037496,0.048268,0.081085,0.0539,0.069787,0.082113,0.166206,0.071697,0.112761,0.145326,0.213569,0.081627,0.090261,0.053519,0.095185,0.082765,0.084917,0.113262,0.147365,0.186231,0.120469,0.128209,0.079564,0.112135,0.160015,0.085764,0.050653,0.085247,0.177404,0.074889,0.087427,0.109489,0.050102,0.06743,0.119345,0.066927,0.054143,0.065293,0.144328,0.078764,0.091914,0.061875,0.04749,0.058924,0.070556,0.065606,0.055345,0.090645,0.081829,0.069899,0.085814,0.083385,0.078677,0.062289,0.087603,0.069681,0.078481,0.104192,0.123871,0.072839,0.11032,0.08828,0.082275,0.088337,0.071536,0.088664,0.073552,0.041301,0.076077,0.09277,0.057026,0.052737,0.051463,0.118503,0.071454,0.122789,0.119352,0.065162,0.